# Back Propagation, Resilient Propagation (RProp) and QuickProp
- Import all the required libraries for the implementation.
- Used 42 as the random seed for the entire notebook. 

In [ ]:
import time
import numpy as np
import pandas as pd
from tabulate import tabulate
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

np.random.seed(42)

### Getting the data and Data preprocessing
- Downloaded the Concrete Compressive Strength dataset from the link below.
- Link: https://archive.ics.uci.edu/dataset/165/concrete+compressive+strength
- Extracted the zip file and placed the Concret_Data.xls file in the directory containing the notebook.
- Loaded the data form the file and split the data into train and test data with train size = 0.7
- Both the dependent and independent data are normalized to make sure data lies within a given range.
- Normalization was done in order to avoid the integer overflow during the weight updates.

In [ ]:
data = pd.read_excel('Concrete_Data.xls')
X = data.iloc[:,0:8]
Y = data.iloc[:,8]
columns = list(X)
for i in columns:
    X[i] = (X[i] - min(X[i])) / (max(X[i]) - min(X[i]))
Y = (Y - min(Y)) / (max(Y) - min(Y))
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.7)

### The Algorithm
- Created a class which takes algorithm, number of layers and activation function as parameters.
- The weights, delta weights and previous gradients are intialised randomly in the class.
- Used Mean Squared Error(MSE) as the loss function and 0.001 as the learning rate.
- Each network is trained for 1e3 epochs irrespective of the convergence is achieved or not.
- Implemented separate methods to update weights incase of BackProp, RProp and QuickProp.

In [ ]:
class Network:
    def __init__(self, algo, layers, activation, rate=0.001):
        self.weights = []
        self.algo = algo
        self.layers = layers
        self.func = activation
        self.rate = rate
        self.train = []
        self.test = []

        if algo == 'RProp':
            for i in range(len(layers)-2):
                w = np.random.rand(layers[i]+1, layers[i+1]+1)
                self.weights.append(w / np.sqrt(layers[i]))
            w = np.random.rand(layers[-2]+1, layers[-1])
            self.weights.append(w / np.sqrt(layers[-2]))
            self.prev = [rate*np.ones_like(w) for w in self.weights]
            self.derivatives = [np.zeros_like(w) for w in self.weights]
        else:
            for i in range(len(layers)-2):
                w = np.random.randn(layers[i]+1, layers[i+1]+1)
                self.weights.append(w / np.sqrt(layers[i]))
            w = np.random.randn(layers[-2]+1, layers[-1])
            self.weights.append(w / np.sqrt(layers[-2]))
            if algo == 'QuickProp':
                self.prev = [rate*np.ones_like(w) for w in self.weights]
                self.derivatives = [np.zeros_like(w) for w in self.weights]

    def activation(self, x):
        if self.func == 'tanh':
            return np.tanh(x)
        if self.func == 'LeakyReLU':
            return np.where(x>0, x, 0.01*x)
        
    def activation_gradient(self, x):
        if self.func == 'tanh':
            return 1-self.activation(x)**2
        if self.func == 'LeakyReLU':
            return np.where(x>0, 1, 0.01)
    
    def calculate_loss(self, y_test, y_pred):
        y_test = np.atleast_2d(y_test)
        y_pred = np.atleast_2d(y_pred)
        mse = np.mean((y_pred-y_test)**2)
        return mse
        
    def predict(self, x, y):
        values = [np.atleast_2d(x)]
        for layer in range(len(self.weights)):
            z = np.dot(values[layer], self.weights[layer])
            values.append(self.activation(z))
        loss = self.calculate_loss(y, values[-1])
        return loss
        
    def fit(self, x_train, y_train, x_test, y_test, epochs=1e3):
        x_train = np.c_[x_train, np.ones((x_train.shape[0]))]
        x_test = np.c_[x_test, np.ones((x_test.shape[0]))]
        for epoch in range(epochs):
            train_loss, test_loss = 0, 0
            for (x, y) in zip(x_train, y_train):
                train_loss += self.fit_instant(x, y)
            train_loss = train_loss / len(x_train)
            self.train.append(train_loss)

            for (x, y) in zip(x_test, y_test):
                test_loss += self.predict(x, y)
            test_loss = test_loss / len(x_test)
            self.test.append(test_loss)
            # print(f'epoch: {1+epoch}, train_error: {train_loss}, test_error: {test_loss}')

    def fit_instant(self, x, y):
        values = [np.atleast_2d(x)]
        # forward pass
        for layer in range(len(self.weights)):
            z = np.dot(values[layer], self.weights[layer])
            values.append(self.activation(z))
        loss = self.calculate_loss(y, values[-1])

        # backward pass
        error = values[-1]-y
        derivatives = [error*self.activation_gradient(values[-1])]
        for layer in range(len(self.weights)-1, 0, -1):
            delta = np.dot(derivatives[-1], self.weights[layer].T)
            delta = delta*self.activation_gradient(values[layer])
            derivatives.append(delta)
        derivatives = derivatives[::-1]
            
        # update the weights
        if self.algo == 'BackProp':
            for layer in range(len(self.weights)):
                self.weights[layer] += -self.rate*np.dot(values[layer].T, derivatives[layer])

        if self.algo == 'RProp':
            alpha = 1.2
            beta = 0.5
            for layer in range(len(self.weights)):
                gradient = np.dot(values[layer].T, derivatives[layer])
                sign = np.sign(gradient)*np.sign(self.derivatives[layer])
                delta = self.prev[layer]
                delta[sign>0] *= alpha
                delta[sign<0] *= beta
                delta = np.clip(delta, 1e-06, 50)
                self.prev[layer] = delta
                self.weights[layer] += -sign*delta
                self.derivatives[layer] = derivatives[layer]

        if self.algo == 'QuickProp':
            for layer in range(len(self.weights)):
                delta = (self.prev[layer]*derivatives[layer]) / (self.derivatives[layer]-derivatives[layer]+1e-9)
                delta = np.clip(delta, 1e-06, 50)
                self.derivatives[layer] = derivatives[layer]
                self.weights[layer] += self.rate*delta
                self.prev[layer] = delta
            
        return loss
    
    def calculate(self):
        threshold = 1e-3
        loss = self.test[-1]
        for epoch, mse in enumerate(self.test):
            if epoch > 0 and abs(mse-self.test[epoch-1]) < threshold:
                return loss, epoch
        return loss, 1000
    
    def lowestMSE(self):
        return min(self.train), min(self.test)
    
    def plot(self, title):
        idx = [i+1 for i in range(1000)]
        plt.figure(figsize=(8, 6))
        plt.plot(idx, self.train, 'b', label='train data')
        plt.plot(idx, self.test, 'r', label='test data')
        plt.xlabel('Number of epochs')
        plt.ylabel('MSE')

        plt.title(title)
        plt.legend()
        plt.show()

### Running the Algorithm
- Used BackPropagation, Resilient Propagation and QuickProp as the three algorithms.
- Used tanh, LeakyReLU as the activation functions and 25 or 50 hidden units in each network.
- Trained each of the triplet (algorithm, number of layers, activation function).
- Graphs of MSE vs number of epochs is plotted for train and test data for each network.
- Also, indicated the loss, epochs to convergence, time for each network in a table.

In [ ]:
algos = ['BackProp', 'QuickProp', 'RProp']
activations = ['tanh', 'LeakyReLU']
layers = [[8, 25, 1], [8, 50, 1]]

results = []
for algo in algos:
    for layer in layers:
        for activation in activations:
            network = Network(algo, layer, activation)
            start = time.time()
            network.fit(x_train, y_train, x_test, y_test, 1000)
            end = time.time()
            title = f'Plot for {algo} using {activation} ({layer[1]} hidden units)'
            network.plot(title)
            l1, l2 = network.lowestMSE()
            loss, epochs = network.calculate()
            results.append([algo, layer[1], activation, loss, epochs, end-start, l1, l2])

headers = ['Network', 'Hidden Units', 'Activation', 'loss', 'epochs to convergence', 'time', 'lowest train MSE', 'lowest test MSE']
print(tabulate(results, headers=headers, tablefmt='grid'))

### The Results
- From the data obtained we can conclude that BackProp takes less time and is optimized algorithm.
- Using 50 hidden layers leads to earlier convergence as compared to 25 hidden layers.
- Leaky ReLU has lower loss in BackProp and QuickProp whereas tanh has lower loss in case of RProp.
- So, the choice of activation function depends on specific problem and the architecture of the network.

![BackPropagation with tanh (25 hidden units)](BPtanh25.png)
![BackPropagation with tanh (50 hidden units)](BPtanh50.png)
![BackPropagation with Leaky ReLU (25 hidden units)](BPLReLU25.png)
![BackPropagation with Leaky ReLU (50 hidden units)](BPLReLU50.png)
![QuickPropagation with tanh (25 hidden units)](QPtanh25.png)
![QuickPropagation with tanh (50 hidden units)](QPtanh50.png)
![QuickPropagation with Leaky ReLU (25 hidden units)](QPLReLU25.png)
![QuickPropagation with Leaky ReLU (50 hidden units)](QPLReLU50.png)
![RPropagation with tanh (25 hidden units)](RPtanh25.png)
![RPropagation with tanh (50 hidden units)](RPtanh50.png)
![RPropagation with Leaky ReLU (25 hidden units)](RPLReLU25.png)
![RPropagation with Leaky ReLU (50 hidden units)](RPLReLU50.png)
![Data Analysis](data.png)